In [1]:
import ast
f = open("data", "r")
data_str = f.read()
data = ast.literal_eval(data_str)

In [2]:
# familiarize with keys if needed 
# key_set = []

# for dictionary in data:
#     for key in dictionary:
#         key_set.append(key)
    
# key_set = set(key_set)
# print key_set



# 'link' and 'caption' has info on the link
# to do: connect with users to find how many places they end up visiting just from fb

# explore key if needed
# type_split = {}
# key = 'link'

# for dictionary in data:
#     if key in dictionary:
#         if dictionary['type'] in type_split:
#             type_split[dictionary['type']].append(dictionary[key])
#         else:
#             type_split[dictionary['type']] = [dictionary[key]]
            
# for x in type_split:
#     print x
#     print "+++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
#     for y in type_split[x]:
#         print y
#         print ""

In [3]:
content = {}
        
for dictionary in data:
    # omit events since they are not reliable sources of information
    if dictionary['type'] not in ('event'):
        
        # user *posting*
        # eventually group by poster to avoid duplication
        user = dictionary['from']['id']
        
        # if the user is not found, add an empty list to the dictionary
        if not user in content:
            content[user] = []
        
        # add description
        if 'description' in dictionary:
            content[user].append(dictionary['description'])
        
        # add message
        if 'message' in dictionary:
            content[user].append(dictionary['message'])
            
        # add name if not in photo, since names of photos are not useful
        if dictionary['type'] not in ('photo') and 'name' in dictionary:
            content[user].append(dictionary['name'])
            
        # add any comments
        if 'comments' in dictionary:
            # each post has a set of comments
            for comment in dictionary['comments']['data']:
                # append the actual comment string
                content[user].append(comment['message']) 
#                 print comment['message']

In [4]:
# Split the text by word, clean it and simplify words into stems

import nltk
import re

# Porter stemmer is a pre-trained stemmer that finds the most likely stem for an english word
from gensim.parsing import PorterStemmer
global_stemmer = PorterStemmer()

# NLTK stopwords to remove (in addition to the scikit-learn default)
from nltk.corpus import stopwords
myStop = stopwords.words('english')
myStop = list(set(myStop) - set(['only', 'has']))
# print myStop

# Remove unicode, normalizing it to ascii
import unicodedata

# Custom tokenizer
def tokenize(doc):
    # Clean the document by normalizing to ascii
    doc = unicode(doc, 'utf-8', errors='ignore')
    doc = unicodedata.normalize('NFKD', doc).encode('ascii','ignore')        

    # Remove hyphens and dashes
    doc = re.sub('[-—]', ' ', doc)
    
    # Strip whitespace
    doc = doc.strip()

    # First tokenize by sentence then by word
    tokens = [word for sent in nltk.sent_tokenize(doc) for word in nltk.word_tokenize(sent)]

    filtered_tokens = []
    # Filter out any tokens not containing at least 2 letters
    for token in tokens:
        if len(re.sub('[^a-zA-Z]', '', token)) > 2:
            # and word not in myStop
            # Stem the token
            filtered_tokens.append(global_stemmer.stem(token))      

    return filtered_tokens

In [5]:
# Find specific filters like pets, male/female only, has parking

In [6]:
# Before doing this, get an idea of common words surrounding the below keywords (stemmed)
main = [global_stemmer.stem(x) for x in ['male', 'female', 'pet', 'parking', 'gender', 'intern']]
print main

keywords = [global_stemmer.stem(x) for x in ['male', 'female', 'pet', 'parking', 'gender', 'intern',
    'furnish',
    'internet',
    'bathroom',
    'utilities',
    'util',
    'laundry',
    'wifi',
    'girl',
    'boy',
    'guy',
    'cat',
    'dog',
    'facility',
    'facilities',
    'gym',
    'private',
    'ac',
    'conditioning',
    'maintenance',
    'kitchen',
    'bus'
    ]]

keywords = list(set(keywords))
print keywords

['male', 'femal', 'pet', 'park', 'gender', 'intern']
['ac', 'intern', 'girl', 'facil', 'privat', 'pet', 'internet', 'condit', 'gym', 'park', 'bathroom', 'util', 'bu', 'mainten', 'laundri', 'kitchen', 'femal', 'gender', 'gui', 'wifi', 'dog', 'cat', 'boi', 'male', 'furnish']


In [7]:
content_tokenized = {}

for user in content:
    content_tokenized[user] = []
    for info in content[user]:
        temp = tokenize(info)
        if len(temp) > 1:
            content_tokenized[user].append(temp)

In [8]:
# could be duplication somewhere

In [9]:
def neighbours(my_list,index,depth):
    lower = max(0, index-depth)
    upper = min(len(my_list), index+1+depth)
    
    before = []
    after = []
    
    if lower < index:
        before = my_list[lower:index]
    if upper > index:
        after = my_list[index+1:upper]
    
    before.extend(after)
    return before

keyword_neighbours = {}
for keyword  in keywords:
    keyword_neighbours[keyword] = []

for user in content_tokenized:
    for content in content_tokenized[user]:
        for i, word in enumerate(content):
            if word in keywords:
                keyword_neighbours[word].append(neighbours(content, i, 2))
                

In [10]:
#verify the word is being used in the desired context most of the time
from collections import Counter
a = Counter()

for keyword in keyword_neighbours:
    print ''
    print '++++++++++++++++++++++++++++++++++++++++++'
    print keyword
    wordCount = {}
    print '++++++++++++++++++++++++++++++++++++++++++'
    for neighbours in keyword_neighbours[keyword]:
        for word in neighbours:
            if word not in wordCount:
                wordCount[word] = 1
            else:
                wordCount[word] = wordCount[word] + 1
    for k, v in Counter(wordCount).most_common(5):
        print '%s: %i' % (k, v)


++++++++++++++++++++++++++++++++++++++++++
ac
++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++
intern
++++++++++++++++++++++++++++++++++++++++++
centr: 1
govern: 1
for: 1
innov: 1

++++++++++++++++++++++++++++++++++++++++++
girl
++++++++++++++++++++++++++++++++++++++++++
all: 15
for: 7
unit: 6
fulli: 6
with: 5

++++++++++++++++++++++++++++++++++++++++++
facil
++++++++++++++++++++++++++++++++++++++++++
laundri: 21
free: 9
park: 8
site: 7
picnic: 6

++++++++++++++++++++++++++++++++++++++++++
bathroom
++++++++++++++++++++++++++++++++++++++++++
with: 76
share: 57
bedroom: 56
ensuit: 54
room: 47

++++++++++++++++++++++++++++++++++++++++++
pet
++++++++++++++++++++++++++++++++++++++++++
friendli: 12
free: 9
and: 7
smoke: 7
fulli: 5

++++++++++++++++++++++++++++++++++++++++++
internet
++++++++++++++++++++++++++++++++++++++++++
speed: 192
high: 168
includ: 167
util: 122
and: 98

++++++++++++++++++++++++++++++++++++++++++
condit
++++++++++++++++++++++++++++

In [22]:
keyword_count = {}
for keyword  in keywords:
    keyword_count[keyword] = 0

for user in content_tokenized:
    all_content = []
    for content in content_tokenized[user]:
        all_content.extend(content)
    
    for keyword in keywords:
        if keyword in all_content:
            keyword_count[keyword] = keyword_count[keyword] + 1

In [23]:
for k, v in Counter(keyword_count).most_common(20):
    print '%s: %i' % (k, v)

furnish: 392
internet: 296
bu: 295
util: 265
laundri: 262
park: 218
bathroom: 185
kitchen: 149
femal: 122
condit: 53
gym: 44
privat: 36
pet: 28
wifi: 28
girl: 27
male: 24
facil: 20
gui: 18
cat: 10
gender: 9


In [13]:
len(content_tokenized)

702

In [14]:
filter_count = 0

for user in content_tokenized:
    all_content = []
    for content in content_tokenized[user]:
        all_content.extend(content)
    
    for keyword in keywords:
        if keyword in all_content:
            filter_count = filter_count + 1
            break
            
print filter_count

575


In [15]:
all_content = []
for user in content_tokenized:
    for content in content_tokenized[user]:
        all_content.extend(content)

all_count = Counter(all_content)

In [16]:
for k, v in all_count.most_common(50):
        if k not in myStop:
            if k not in ['room', 'includ', 'waterloo', 'avail', 'bedroom', 'look', 'bu', 'ontario', 'pleas'
                        , 'month', 'august', 'mai', 'price', 'unit', 'wlu']:
                print '%s: %i' % (k, v)

sublet: 762
walk: 748
furnish: 670
internet: 485
minut: 430
fulli: 429
laundri: 389
util: 384
spring: 377
messag: 376
bathroom: 350
min: 337
park: 332
close: 327
free: 297
hous: 290
kitchen: 276
locat: 271
leas: 262
street: 258
stop: 255
high: 246
negoti: 235
inclus: 235
larg: 230
ensuit: 228
summer: 221


In [17]:
filter_count = 0

for user in content_tokenized:
    all_content = []
    for content in content_tokenized[user]:
        all_content.extend(content)
    
    for keyword in main:
        if keyword in all_content:
            filter_count = filter_count + 1
            break
            
print filter_count

331


In [18]:
keyword_count = {}
for keyword  in main:
    keyword_count[keyword] = 0

for user in content_tokenized:
    all_content = []
    for content in content_tokenized[user]:
        all_content.extend(content)
    
    for keyword in main:
        if keyword in all_content:
            keyword_count[keyword] = keyword_count[keyword] + 1

In [19]:
keyword_count

{'femal': 122, 'gender': 9, 'intern': 1, 'male': 24, 'park': 218, 'pet': 28}